# SEIR-Modell angepasst

In [32]:
import numpy as np
from scipy.integrate import ode
import pandas as pd
import matplotlib.pyplot as pl
import matplotlib.patches as patches
pd.set_option('display.max_rows', None)
import altair as alt
import matplotlib.pyplot as plt
import matplotlib.colors

This is the function that is used to integrate over time

In [33]:
# set equation of motion for SIR dynamics
def dxdt(t,y,eta,rho,epsilon):

    S = y[0]
    I = y[1]
    R = y[2]
    E = y[3]

    #_eta = eta*(1.03+0.5*np.sin(np.pi*2*t/28))
    _eta = eta

    dy = np.zeros(4)
    dy[0] = -_eta*S*I
    dy[1] = +epsilon*E - rho*I
    dy[2] = +rho*I
    dy[3] = +_eta*S*I - epsilon*E

    return dy

These are the parameters to adjust the model roughly for the circumstances in Germany and for how long we want to run it

https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Modellierung_Deutschland.pdf?__blob=publicationFile

In [34]:
# recovery rate
infectious_duration = 10 #days
rho = 1/infectious_duration

# infection rate
R0 = 2.5 # basic reproduction number
eta = R0*rho # mean-field 


# asymptomatic period
asymptomatic_duration = 3 #days
epsilon = 1/asymptomatic_duration

# number of people in Germany
N = 8.2e7

# initially infected (think, people who came from italy, austria etc. infected)
initial = 1000

# max value of time and points in time to integrate to
t_max = 365 #days
N_spacing_in_t = t_max


t_0 = 0 # intitial time

# create vector of time points you want to evaluate
t = np.linspace(t_0,t_max,N_spacing_in_t)

And now we want to try out how different interventions with different severities impact SEIR and also the share of people in ICU

In [35]:

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#ffffff","#0570b0"])


In [36]:
maxpoint=0.2


# This is for changing the effectiveness during intervention

In [37]:
for i in range(10,110,10):
# days after start after which action is effective
    action_taken_at_day = 30

    # how many days does the action last?
    action_duration = 28

    # fraction of people seen during action 
    action_effectiveness = i/100
    post_action_effectiveness =100/100
    #we reproject parameters to t
    action_taken_in_t = int(N_spacing_in_t / t_max * action_taken_at_day)
    action_duration_in_t = int(N_spacing_in_t / t_max * action_duration)

    #and we adjust the infection rate during the intervention
    R_action = R0*action_effectiveness # basic reproduction number during intervention
    eta_action = R0*rho*action_effectiveness # mean-field during intervention


    # nd we adjust infection rate after the intervention
    R_post_action = R0*post_action_effectiveness # basic reproduction number during intervention
    eta_post_action = R0*rho*post_action_effectiveness # mean-field during intervention


    # create vector of positions for those times
    result = np.zeros((4,len(t)))
    # initial values, before action
    I_0 = initial / N
    E_0 = 0 
    S_0 = 1 - I_0 # initially susceptible
    R_0 = 0

    # initial y-vector
    y0 = np.array([S_0,I_0,R_0,E_0])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y0,t_0)

    # set transmission rate and recovery rate
    r.set_f_params(eta,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[:action_taken_in_t]):

        # get result of ODE integration
        y = r.integrate(t_)
        # write result to result vector

        result[:,it] = y
    # setup second model with adjusted parameters (start of action)
    S_1, I_1, R_1, E_1 = result[:,action_taken_in_t - 1]

    # initial y-vector when action is taken
    y1 = np.array([S_1,I_1,R_1,E_1])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y1,t[action_taken_in_t-1])

    # set transmission rate and recovery rate, adjust R_0 according to our effectiveness
    r.set_f_params(eta_action,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[action_taken_in_t:action_taken_in_t+action_duration_in_t], start=action_taken_in_t):
        # get result of ODE integration
        y = r.integrate(t_)
        # write result to result vector
        result[:,it] = y
    # setup third model and run it until t_max
    S_2, I_2, R_2, E_2 = result[:,action_taken_in_t + action_duration_in_t - 1]

    # initial y-vector
    y1 = np.array([S_2,I_2,R_2,E_2])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y1,t[action_taken_in_t + action_duration_in_t-1])

    # set transmission rate and recovery rate, adjust R_0 according to our effectiveness
    r.set_f_params(eta_post_action,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[action_taken_in_t + action_duration_in_t:], start=action_taken_in_t + action_duration_in_t):

        # get result of ODE integration
        y = r.integrate(t_)

        # write result to result vector
        result[:,it] = y
    results_df=pd.DataFrame(result).transpose()
    results_df.columns=["Susceptible","Infected","Recovered","Exposed"]
    results_df=results_df.reset_index().rename(columns={"index":"Tage"})
    results_df
    results_df["time_during_action"]=0
    results_df["time_during_action"].loc[action_taken_at_day:(action_taken_at_day+action_duration)]=maxpoint*1.1
    results_df["time_after_action"]=0
    results_df["time_after_action"].loc[(action_taken_at_day+action_duration):]=maxpoint*1.1
    results_df
    #here we just transform the data to longform for altair to deal with it
    longform=pd.melt(results_df, id_vars=['Tage'], value_vars=['Susceptible',"Exposed",'Infected',"Recovered","time_during_action","time_after_action",],value_name='Anteil der Bevölkerung')
    longform


    #x,y,c = zip(*np.random.rand(30,3)*4-2)

    #cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#3fbaed","#2887f0"])


    #Generate a color scale
    #cmap = matplotlib.cm.get_cmap('Spectral')

    #Select the color 75% of the way through the colorscale   
    rgba = cmap((1-action_effectiveness))
    print(rgba)

    val1=matplotlib.colors.to_hex(rgba)
    rgba = cmap((1-post_action_effectiveness))
    print(rgba)

    val2=matplotlib.colors.to_hex(rgba)
    chart1=alt.Chart(longform[longform["variable"].isin(["Infected"])]).mark_line(color='orange'
        ).encode(
        alt.X('Tage:Q',scale=alt.Scale(
            domain=(0, 366),
            clamp=True
        )),
        alt.Y('Anteil der Bevölkerung:Q',axis=alt.Axis(format='%')),
        tooltip=['Tage', 'Anteil der Bevölkerung'])

    chart1
    chart2=alt.Chart(longform[longform["variable"].isin([ "time_after_action"])]).mark_area(
        interpolate='step-after',color=val2, opacity=0.5).encode(
        alt.X('Tage:Q',scale=alt.Scale(
            domain=(0, 366),
            clamp=True
        )),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
            clamp=True
        ),axis=alt.Axis(format='%')),
        )


    chart2
    chart3=alt.Chart(longform[longform["variable"].isin([ "time_during_action"])]).mark_area(
        interpolate='step-before',
        color=val1,opacity=0.5).encode(
        alt.X('Tage:Q',scale=alt.Scale(
            domain=(0, 366),
            clamp=True
        )),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
            clamp=True
        ),axis=alt.Axis(format='%'))
        )


    baked=chart3+chart2+chart1
    baked.display()
    baked.save("../Viz/share_of_r0_during_intervention/"+str(i)+'.svg')


(0.11572472126105349, 0.4941945405613225, 0.7205690119184929, 1.0)
(1.0, 1.0, 1.0, 1.0)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

(0.21568627450980393, 0.5513725490196077, 0.7521568627450981, 1.0)
(1.0, 1.0, 1.0, 1.0)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


...

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

(1.0, 1.0, 1.0, 1.0)
(1.0, 1.0, 1.0, 1.0)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

# This is for increasing the timespan of intervention

In [38]:
for i in range(4,9,1):
# days after start after which action is effective
    action_taken_at_day = 30

    # how many days does the action last?
    action_duration = i*7

    # fraction of people seen during action 
    action_effectiveness = 50/100
    post_action_effectiveness =100/100
    #we reproject parameters to t
    action_taken_in_t = int(N_spacing_in_t / t_max * action_taken_at_day)
    action_duration_in_t = int(N_spacing_in_t / t_max * action_duration)

    #and we adjust the infection rate during the intervention
    R_action = R0*action_effectiveness # basic reproduction number during intervention
    eta_action = R0*rho*action_effectiveness # mean-field during intervention


    # nd we adjust infection rate after the intervention
    R_post_action = R0*post_action_effectiveness # basic reproduction number during intervention
    eta_post_action = R0*rho*post_action_effectiveness # mean-field during intervention


    # create vector of positions for those times
    result = np.zeros((4,len(t)))
    # initial values, before action
    I_0 = initial / N
    E_0 = 0 
    S_0 = 1 - I_0 # initially susceptible
    R_0 = 0

    # initial y-vector
    y0 = np.array([S_0,I_0,R_0,E_0])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y0,t_0)

    # set transmission rate and recovery rate
    r.set_f_params(eta,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[:action_taken_in_t]):

        # get result of ODE integration
        y = r.integrate(t_)
        # write result to result vector

        result[:,it] = y
    # setup second model with adjusted parameters (start of action)
    S_1, I_1, R_1, E_1 = result[:,action_taken_in_t - 1]

    # initial y-vector when action is taken
    y1 = np.array([S_1,I_1,R_1,E_1])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y1,t[action_taken_in_t-1])

    # set transmission rate and recovery rate, adjust R_0 according to our effectiveness
    r.set_f_params(eta_action,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[action_taken_in_t:action_taken_in_t+action_duration_in_t], start=action_taken_in_t):
        # get result of ODE integration
        y = r.integrate(t_)
        # write result to result vector
        result[:,it] = y
    # setup third model and run it until t_max
    S_2, I_2, R_2, E_2 = result[:,action_taken_in_t + action_duration_in_t - 1]

    # initial y-vector
    y1 = np.array([S_2,I_2,R_2,E_2])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y1,t[action_taken_in_t + action_duration_in_t-1])

    # set transmission rate and recovery rate, adjust R_0 according to our effectiveness
    r.set_f_params(eta_post_action,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[action_taken_in_t + action_duration_in_t:], start=action_taken_in_t + action_duration_in_t):

        # get result of ODE integration
        y = r.integrate(t_)

        # write result to result vector
        result[:,it] = y
    results_df=pd.DataFrame(result).transpose()
    results_df.columns=["Susceptible","Infected","Recovered","Exposed"]
    results_df=results_df.reset_index().rename(columns={"index":"Tage"})
    results_df
    results_df["time_during_action"]=0
    results_df["time_during_action"].loc[action_taken_at_day:(action_taken_at_day+action_duration)]=maxpoint
    results_df["time_after_action"]=0
    results_df["time_after_action"].loc[(action_taken_at_day+action_duration):]=maxpoint
    results_df
    #here we just transform the data to longform for altair to deal with it
    longform=pd.melt(results_df, id_vars=['Tage'], value_vars=['Susceptible',"Exposed",'Infected',"Recovered","time_during_action","time_after_action",],value_name='Anteil der Bevölkerung')
    longform
 

    #x,y,c = zip(*np.random.rand(30,3)*4-2)



    #Generate a color scale
    #cmap = matplotlib.cm.get_cmap('Spectral')

    #Select the color 75% of the way through the colorscale   
    rgba = cmap((1-action_effectiveness))

    val1=matplotlib.colors.to_hex(rgba)
    rgba = cmap((1-post_action_effectiveness))
    val2=matplotlib.colors.to_hex(rgba)
    chart1=alt.Chart(longform[longform["variable"].isin(["Infected"])]).mark_line(color='orange'
        ).encode(
        alt.X('Tage:Q'),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
        ),axis=alt.Axis(format='%')),
        tooltip=['Tage', 'Anteil der Bevölkerung'])

    chart1
    chart2=alt.Chart(longform[longform["variable"].isin([ "time_after_action"])]).mark_area(
        interpolate='step-after',color=val2, opacity=0.5).encode(
        alt.X('Tage:Q'),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
        ),axis=alt.Axis(format='%'))
    )


    chart2
    chart3=alt.Chart(longform[longform["variable"].isin([ "time_during_action"])]).mark_area(
        interpolate='step-before',
        color=val1,opacity=0.5).encode(
        alt.X('Tage:Q'),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
        ),axis=alt.Axis(format='%'))
        )


    baked=chart3+chart2+chart1
    baked.display()
    baked.save("../Viz/duration_of_intervention/"+str(i)+'.svg')


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

# Now zoom in on what has happened during the intervention

In [46]:
maxpoint=0.0025
# recovery rate
infectious_duration = 10 #days
rho = 1/infectious_duration

# infection rate
R0 = 2.5 # basic reproduction number
eta = R0*rho # mean-field 


# asymptomatic period
asymptomatic_duration = 3 #days
epsilon = 1/asymptomatic_duration

# number of people in Germany
N = 8.2e7

# initially infected (think, people who came from italy, austria etc. infected)
initial = 1000

# max value of time and points in time to integrate to
t_max = 58 #days
N_spacing_in_t = t_max


t_0 = 0 # intitial time

# create vector of time points you want to evaluate
t = np.linspace(t_0,t_max,N_spacing_in_t)

In [48]:
for i in range(10,110,10):
# days after start after which action is effective
    action_taken_at_day = 30

    # how many days does the action last?
    action_duration = 28

    # fraction of people seen during action 
    action_effectiveness = i/100
    post_action_effectiveness =100/100


    #we reproject parameters to t
    action_taken_in_t = int(N_spacing_in_t / t_max * action_taken_at_day)
    action_duration_in_t = int(N_spacing_in_t / t_max * action_duration)

    #and we adjust the infection rate during the intervention
    R_action = R0*action_effectiveness # basic reproduction number during intervention
    eta_action = R0*rho*action_effectiveness # mean-field during intervention


    # nd we adjust infection rate after the intervention
    R_post_action = R0*post_action_effectiveness # basic reproduction number during intervention
    eta_post_action = R0*rho*post_action_effectiveness # mean-field during intervention


    # create vector of positions for those times
    result = np.zeros((4,len(t)))
    # initial values, before action
    I_0 = initial / N
    E_0 = 0 
    S_0 = 1 - I_0 # initially susceptible
    R_0 = 0

    # initial y-vector
    y0 = np.array([S_0,I_0,R_0,E_0])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y0,t_0)

    # set transmission rate and recovery rate
    r.set_f_params(eta,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[:action_taken_in_t]):

        # get result of ODE integration
        y = r.integrate(t_)
        # write result to result vector

        result[:,it] = y
    # setup second model with adjusted parameters (start of action)
    S_1, I_1, R_1, E_1 = result[:,action_taken_in_t - 1]

    # initial y-vector when action is taken
    y1 = np.array([S_1,I_1,R_1,E_1])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y1,t[action_taken_in_t-1])

    # set transmission rate and recovery rate, adjust R_0 according to our effectiveness
    r.set_f_params(eta_action,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[action_taken_in_t:action_taken_in_t+action_duration_in_t], start=action_taken_in_t):
        # get result of ODE integration
        y = r.integrate(t_)
        # write result to result vector
        result[:,it] = y

    results_df=pd.DataFrame(result).transpose()
    results_df.columns=["Susceptible","Infected","Recovered","Exposed"]
    results_df=results_df.reset_index().rename(columns={"index":"Tage"})
    results_df
    results_df["time_during_action"]=0
    results_df["time_during_action"].loc[action_taken_at_day:(action_taken_at_day+action_duration)]=maxpoint
    results_df["time_after_action"]=0
    results_df["time_after_action"].loc[(action_taken_at_day+action_duration):]=maxpoint
    results_df
    #here we just transform the data to longform for altair to deal with it
    longform=pd.melt(results_df, id_vars=['Tage'], value_vars=['Susceptible',"Exposed",'Infected',"Recovered","time_during_action","time_after_action",],value_name='Anteil der Bevölkerung')
    longform
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.colors

    #x,y,c = zip(*np.random.rand(30,3)*4-2)



    #Generate a color scale
    #cmap = matplotlib.cm.get_cmap('Spectral')

    #Select the color 75% of the way through the colorscale   
    rgba = cmap((1-action_effectiveness))

    val1=matplotlib.colors.to_hex(rgba)
    rgba = cmap((1-post_action_effectiveness))
    val2=matplotlib.colors.to_hex(rgba)
    chart1=alt.Chart(longform[longform["variable"].isin(["Infected"])]).mark_line(color='orange',clip=True
        ).encode(
        alt.X('Tage:Q'),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
            clamp=True
        ),axis=alt.Axis(format='%')),
        tooltip=['Tage', 'Anteil der Bevölkerung'])

    chart1
    chart2=alt.Chart(longform[longform["variable"].isin([ "time_after_action"])]).mark_area(clip=True,
        interpolate='step-after',color=val2, opacity=0.5).encode(
        alt.X('Tage:Q'),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
            clamp=True
        ),axis=alt.Axis(format='%'))
    )


    chart2
    chart3=alt.Chart(longform[longform["variable"].isin([ "time_during_action"])]).mark_area(clip=True,
        interpolate='step-before',
        color=val1,opacity=0.5).encode(
        alt.X('Tage:Q'),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
            clamp=True
        ),axis=alt.Axis(format='%'))
        )

    print(i)
    baked=chart3+chart2+chart1
    baked.display()
    baked.save("../Viz/duration_of_intervention_zoomed/"+str(i)+'.svg')

10


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

20


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


...

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

100


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

# What happens now after the intervention depending on R0

In [41]:
# recovery rate
infectious_duration = 10 #days
rho = 1/infectious_duration

# infection rate
R0 = 2.5 # basic reproduction number
eta = R0*rho # mean-field 


# asymptomatic period
asymptomatic_duration = 3 #days
epsilon = 1/asymptomatic_duration

# number of people in Germany
N = 8.2e7

# initially infected (think, people who came from italy, austria etc. infected)
initial = 1000

# max value of time and points in time to integrate to
t_max = 365 #days
N_spacing_in_t = t_max


t_0 = 0 # intitial time

# create vector of time points you want to evaluate
t = np.linspace(t_0,t_max,N_spacing_in_t)

In [42]:
maxpoint=0.2


In [45]:
for i in range(10,110,10):
# days after start after which action is effective
    action_taken_at_day = 30

    # how many days does the action last?
    action_duration = 28

    # fraction of people seen during action 
    action_effectiveness = 50/100
    post_action_effectiveness =i/100
    #we reproject parameters to t
    action_taken_in_t = int(N_spacing_in_t / t_max * action_taken_at_day)
    action_duration_in_t = int(N_spacing_in_t / t_max * action_duration)

    #and we adjust the infection rate during the intervention
    R_action = R0*action_effectiveness # basic reproduction number during intervention
    eta_action = R0*rho*action_effectiveness # mean-field during intervention


    # nd we adjust infection rate after the intervention
    R_post_action = R0*post_action_effectiveness # basic reproduction number during intervention
    eta_post_action = R0*rho*post_action_effectiveness # mean-field during intervention


    # create vector of positions for those times
    result = np.zeros((4,len(t)))
    # initial values, before action
    I_0 = initial / N
    E_0 = 0 
    S_0 = 1 - I_0 # initially susceptible
    R_0 = 0

    # initial y-vector
    y0 = np.array([S_0,I_0,R_0,E_0])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y0,t_0)

    # set transmission rate and recovery rate
    r.set_f_params(eta,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[:action_taken_in_t]):

        # get result of ODE integration
        y = r.integrate(t_)
        # write result to result vector

        result[:,it] = y
    # setup second model with adjusted parameters (start of action)
    S_1, I_1, R_1, E_1 = result[:,action_taken_in_t - 1]

    # initial y-vector when action is taken
    y1 = np.array([S_1,I_1,R_1,E_1])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y1,t[action_taken_in_t-1])

    # set transmission rate and recovery rate, adjust R_0 according to our effectiveness
    r.set_f_params(eta_action,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[action_taken_in_t:action_taken_in_t+action_duration_in_t], start=action_taken_in_t):
        # get result of ODE integration
        y = r.integrate(t_)
        # write result to result vector
        result[:,it] = y
    # setup third model and run it until t_max
    S_2, I_2, R_2, E_2 = result[:,action_taken_in_t + action_duration_in_t - 1]

    # initial y-vector
    y1 = np.array([S_2,I_2,R_2,E_2])

    # initialize integrator
    r = ode(dxdt)

    # Runge-Kutta with step size control
    r.set_integrator('dopri5')

    # set initial values
    r.set_initial_value(y1,t[action_taken_in_t + action_duration_in_t-1])

    # set transmission rate and recovery rate, adjust R_0 according to our effectiveness
    r.set_f_params(eta_post_action,rho,epsilon)

    # loop through all demanded time points until action is taken
    for it, t_ in enumerate(t[action_taken_in_t + action_duration_in_t:], start=action_taken_in_t + action_duration_in_t):

        # get result of ODE integration
        y = r.integrate(t_)

        # write result to result vector
        result[:,it] = y
    results_df=pd.DataFrame(result).transpose()
    results_df.columns=["Susceptible","Infected","Recovered","Exposed"]
    results_df=results_df.reset_index().rename(columns={"index":"Tage"})
    results_df
    results_df["time_during_action"]=0
    results_df["time_during_action"].loc[action_taken_at_day:(action_taken_at_day+action_duration)]=maxpoint*1.1
    results_df["time_after_action"]=0
    results_df["time_after_action"].loc[(action_taken_at_day+action_duration):]=maxpoint*1.1
    results_df
    #here we just transform the data to longform for altair to deal with it
    longform=pd.melt(results_df, id_vars=['Tage'], value_vars=['Susceptible',"Exposed",'Infected',"Recovered","time_during_action","time_after_action",],value_name='Anteil der Bevölkerung')
    longform


    #x,y,c = zip(*np.random.rand(30,3)*4-2)

    #cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#3fbaed","#2887f0"])


    #Generate a color scale
    #cmap = matplotlib.cm.get_cmap('Spectral')

    #Select the color 75% of the way through the colorscale   
    rgba = cmap((1-action_effectiveness))
    print(rgba)

    val1=matplotlib.colors.to_hex(rgba)
    rgba = cmap((1-post_action_effectiveness))
    print(rgba)

    val2=matplotlib.colors.to_hex(rgba)
    chart1=alt.Chart(longform[longform["variable"].isin(["Infected"])]).mark_line(color='orange'
        ).encode(
        alt.X('Tage:Q',scale=alt.Scale(
            domain=(0, 366),
            clamp=True
        )),
        alt.Y('Anteil der Bevölkerung:Q',axis=alt.Axis(format='%')),
        tooltip=['Tage', 'Anteil der Bevölkerung'])

    chart1
    chart2=alt.Chart(longform[longform["variable"].isin([ "time_after_action"])]).mark_area(
        interpolate='step-after',color=val2, opacity=0.5).encode(
        alt.X('Tage:Q',scale=alt.Scale(
            domain=(0, 366),
            clamp=True
        )),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
            clamp=True
        ),axis=alt.Axis(format='%')),
        )


    chart2
    chart3=alt.Chart(longform[longform["variable"].isin([ "time_during_action"])]).mark_area(
        interpolate='step-before',
        color=val1,opacity=0.5).encode(
        alt.X('Tage:Q',scale=alt.Scale(
            domain=(0, 366),
            clamp=True
        )),
        alt.Y('Anteil der Bevölkerung:Q',scale=alt.Scale(
            domain=(0, maxpoint),
            clamp=True
        ),axis=alt.Axis(format='%'))
        )


    baked=chart3+chart2+chart1
    baked.display()
    baked.save("../Viz/share_of_r0_after_intervention/"+str(i)+'.svg')


(0.5078815840061515, 0.7185082660515186, 0.8444905805459438, 1.0)
(0.11572472126105349, 0.4941945405613225, 0.7205690119184929, 1.0)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

(0.5078815840061515, 0.7185082660515186, 0.8444905805459438, 1.0)
(0.21568627450980393, 0.5513725490196077, 0.7521568627450981, 1.0)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


...

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)

(0.5078815840061515, 0.7185082660515186, 0.8444905805459438, 1.0)
(1.0, 1.0, 1.0, 1.0)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/integrate/_ode.py:1177: UserWarning: dopri5: step size becomes too small
  self.messages.get(istate, unexpected_istate_msg)))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


alt.LayerChart(...)